In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense # type: ignore
from tensorflow.keras.models import Model #type: ignore
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
df = pd.read_csv('data.csv')
df = df.drop(columns=['Unnamed: 0']) 

In [3]:
le_start = LabelEncoder()
le_dest = LabelEncoder()
df['start_pin'] = le_start.fit_transform(df['start_pin'])
df['destination_pin'] = le_dest.fit_transform(df['destination_pin'])

In [4]:
X = df.drop(columns=['amount'])
y = df['amount']

In [5]:
print(X.head(), y.head())

   start_pin  destination_pin  travel_distance  Quantity (In TON)
0         28             2338             1115               75.0
1         40              231              367               13.0
2         40              255              493               12.0
3         40              323              304               40.0
4         41              216              306               39.4 0    585600.00
1    104260.01
2     60314.40
3    272000.00
4    299440.50
Name: amount, dtype: float64


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
num_features = ['travel_distance', 'Quantity (In TON)']
scaler = StandardScaler()
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])

In [8]:
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))


In [9]:
num_start_pins = df['start_pin'].nunique()
num_dest_pins = df['destination_pin'].nunique()
embedding_dim = 64

In [10]:
start_embed_dim = min(8, int(np.power(num_start_pins, 0.25)))
dest_embed_dim = min(16, int(np.power(num_dest_pins, 0.25)))

In [11]:
start_pin_input = Input(shape=(1,), name='start_pin')
dest_pin_input = Input(shape=(1,), name='destination_pin')
numerical_input = Input(shape=(2,), name='numerical_data')

In [12]:
start_embedding = Embedding(input_dim=num_start_pins + 1, output_dim=start_embed_dim)(start_pin_input)
dest_embedding = Embedding(input_dim=num_dest_pins + 1,output_dim=dest_embed_dim)(dest_pin_input)

In [13]:
start_embedding

<KerasTensor: shape=(None, 1, 6) dtype=float32 (created by layer 'embedding')>

In [14]:
start_flat = Flatten()(start_embedding)
dest_flat  = Flatten()(dest_embedding)

In [15]:
x = Concatenate()([start_flat, dest_flat, numerical_input])

In [16]:
# Increase model complexity for better feature learning
x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
output = Dense(1)(x)

In [17]:

x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)
# x = Dense(32,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dropout(0.2)(x)
# x = Dense(16,activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.Dropout(0.2)(x)
output = Dense(1)(x)

In [18]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [19]:
model = Model(inputs=[start_pin_input, dest_pin_input, numerical_input], outputs=output)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start_pin (InputLayer)      [(None, 1)]                  0         []                            
                                                                                                  
 destination_pin (InputLaye  [(None, 1)]                  0         []                            
 r)                                                                                               
                                                                                                  
 embedding (Embedding)       (None, 1, 6)                 8262      ['start_pin[0][0]']           
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 9)                 63999     ['destination_pin[0][0]'] 

In [21]:
train_inputs = [
    X_train['start_pin'].values,               
    X_train['destination_pin'].values,        
    X_train[num_features].values             
]

In [22]:
test_inputs = [
    X_test['start_pin'].values,
    X_test['destination_pin'].values,
    X_test[num_features].values
]

In [23]:
history = model.fit(
    train_inputs, 
    y_train_scaled,
    validation_data=(test_inputs, y_test_scaled),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/100
772/772 [==============================] - 8s 8ms/step - loss: 3.2225 - val_loss: 2.6522
Epoch 2/100
772/772 [==============================] - 8s 10ms/step - loss: 2.1642 - val_loss: 1.9895
Epoch 3/100
772/772 [==============================] - 3s 4ms/step - loss: 1.6148 - val_loss: 1.5236
Epoch 4/100
772/772 [==============================] - 3s 4ms/step - loss: 1.3038 - val_loss: 1.3387
Epoch 5/100
772/772 [==============================] - 2s 3ms/step - loss: 1.1627 - val_loss: 1.2101
Epoch 6/100
772/772 [==============================] - 2s 2ms/step - loss: 1.0880 - val_loss: 1.1715
Epoch 7/100
772/772 [==============================] - 2s 2ms/step - loss: 1.0546 - val_loss: 1.1446
Epoch 8/100
772/772 [==============================] - 2s 2ms/step - loss: 1.0204 - val_loss: 1.1341
Epoch 9/100
772/772 [==============================] - 2s 3ms/step - loss: 1.0216 - val_loss: 1.1171
Epoch 10/100
772/772 [==============================] - 3s 3ms/step - loss: 1.0144 - val_l

In [24]:
predictions = model.predict(test_inputs)
predictions_original = y_scaler.inverse_transform(predictions)

193/193 [==============================] - 0s 684us/step


In [25]:
predictions

array([[-0.0437351],
       [-0.0437351],
       [-0.0437351],
       ...,
       [-0.0437351],
       [-0.0437351],
       [-0.0437351]], dtype=float32)

In [26]:
predictions_original

array([[186669.77],
       [186669.77],
       [186669.77],
       ...,
       [186669.77],
       [186669.77],
       [186669.77]], dtype=float32)

In [27]:
mse = mean_squared_error(y_test_scaled, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_scaled, predictions)
print("MSE:", mse)
print("RMSE:", rmse)
print("R^2:", r2)

MSE: 0.5933553147091581
RMSE: 0.7702956021613768
R^2: 0.4851406545888548
